In [1]:
import concurrent
import cv2
import glob
import matplotlib.pyplot as plt
from matplotlib import image
from matplotlib.patches import Circle, Rectangle
import multiprocessing as mp
import numba
import numpy as np
import os
import re
import seaborn as sns
from scipy import spatial
from scipy import signal
from scipy import ndimage
from scipy import optimize

%matplotlib qt

In [2]:
def gen_target(w: float, h: float, dpi: int, loc: str, plot=False):
    '''
    Generate an image of a photogrammetry target.

    Parameters
    ----------
    w
        total image width, m
    h
        total image height, m
    dpi
        output image dpi, converts between matrix coordinates and real width
    loc
        [SW, NW, SE, NE, raft] generates a different pattern for each loc
    '''
    IN_TO_METERS = 0.0254
    w_in = w / IN_TO_METERS
    h_in = h / IN_TO_METERS
    w_px = np.round(w_in * dpi).astype(int)
    h_px = np.round(h_in * dpi).astype(int)

    fig, ax = plt.subplots()
    fig.set_size_inches(w_in, h_in)
    fig.tight_layout()
    fig.subplots_adjust(left=0., right=1., top=1., bottom=0.)

    pattern_size = 8
    pattern = np.random.choice([0.05,0.95], size=(pattern_size,pattern_size))
    pattern[0][0] = 0
    pattern[pattern_size - 1][pattern_size - 1] = 1
    ax.imshow(ndimage.zoom(pattern, h_px / pattern_size, order=0), cmap='Greys')

    # center cross
    c_size = dpi * 0.01 / IN_TO_METERS # 10 cm
    c_thick = dpi * 0.001 / IN_TO_METERS # 1 mm

    ax.add_patch(Rectangle((w_px / 2 - c_size / 2, h_px / 2 - c_thick / 2), c_size, c_thick, color='silver')) # horz
    ax.add_patch(Rectangle((w_px / 2 - c_thick / 2, h_px / 2 - c_size / 2), c_thick, c_size, color='silver')) # vert
    ax.add_patch(Rectangle((w_px / 2 - c_size / 2, h_px / 2 - c_thick / 4), c_size, c_thick / 2, color='k')) # horz
    ax.add_patch(Rectangle((w_px / 2 - c_thick / 4, h_px / 2 - c_size / 2), c_thick / 2, c_size, color='k')) # vert

    ax.set_aspect('equal')
    ax.set_xlim([0, w_px])
    ax.set_ylim([0, h_px])
    ax.set_xticks([])
    ax.set_yticks([])
    fig.savefig(f'{loc}_target.png')
    if not plot:
        plt.close()

In [3]:
# only run this if you need to re-generate photogrammetry targets.
np.random.seed(77777) # generate same noise pattern every time
gen_locs = ['SW', 'NW', 'NE', 'SE', 'raft']
TARGET_W = 0.03 # 30 mm
TARGET_H = TARGET_W
# for loc in gen_locs:
    # gen_target(TARGET_W, TARGET_H, 300, loc, plot=False)

TARGET_W_AS_PRINTED = 0.03

In [4]:
def calibrate_camera(image_dir, plot=False):
    # https://learnopencv.com/camera-calibration-using-opencv/
    '''
    The general idea is to take a set of test images with a chessboard pattern
    (6x9, i.e. 5x8 intersections) to calculate the distortion parameters of the
    camera/lens combo being used. This will be used to undo distortion effects
    before we can undo projection effects.
    '''
    def find_corners(file):
        im = cv2.imread(file)
        im = np.rot90(im, k=-1) # may not need this, depending on source of images.
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, CHESSBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
        print(f'Camera calibration corner finding in {file}:', ret)
        return ret, corners, im, gray


    objpoints_q = mp.Queue() # the chessboard vertex locations in the plane of the board
    imgpoints_q = mp.Queue() # the chessboard vertex locations in the image space
    
    prev_img_shape = None

    files = (
        sorted(glob.glob(os.path.join(image_dir, '**.jpeg'))) +
        sorted(glob.glob(os.path.join(image_dir, '**.jpg'))) + 
        sorted(glob.glob(os.path.join(image_dir, '**.JPG')))
    )

    # do camera calibration from chessboard images
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as pool:
        future_to_file = {pool.submit(find_corners, file) : file for file in files}
        for future in concurrent.futures.as_completed(future_to_file):
            file = future_to_file[future]
            try:
                ret, corners, im, gray = future.result()
            except Exception as e:
                print(f'file {file} generated an exception: {e}')
            else:
                if ret:
                    objpoints_q.put(objp)
                    corners2 = cv2.cornerSubPix(gray, corners, (5,5),(-1,-1), criteria)
                    imgpoints_q.put(corners2)
                    im = cv2.drawChessboardCorners(gray, CHESSBOARD, corners2, ret)
                if plot:
                    plt.figure()
                    plt.imshow(im)
                    ax = plt.gca()
                    ax.set_aspect('equal')

    # Do the actual camera calibration with all the data we gathered.
    # OpenCV likes lists.
    objpoints = []
    while not objpoints_q.empty():
        objpoints.append(objpoints_q.get(timeout=0.1))
    imgpoints = []
    while not imgpoints_q.empty():
        imgpoints.append(imgpoints_q.get(timeout=0.1))

    h,w = im.shape[:2]
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    optimal_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(
        mtx,
        dist, 
        (w,h), 
        1, 
        (w,h)
    )

    return mtx, dist, optimal_camera_matrix

In [5]:
@numba.jit(nopython=True)
def xcorr_prep(im):
    '''
    Helper function to apply pre-processing that makes cross-correlation work
    '''
    im_corr = np.sum(im.astype(np.float64), axis=2) # ensure single channel
    im_corr /= im_corr.max() # normalize
    im_corr -= np.mean(im_corr) # detrend
    return im_corr

In [17]:
def find_template(template_filename, im_warped, avg_px_per_m, ax=None):
    '''
    Returns the pixel coordinates of the best-cross-corr match of the template image in im_warped,
    which ought to be a lens-distortion-free, camera-angle-projection-free image.
    avg_px_per_m is needed to scale the target's known size in m to the image's pixel scale.
    '''

    im_tmp = cv2.imread(template_filename)
    # number of pixels in a shape with some real size in meters:
    target_px = avg_px_per_m * TARGET_W_AS_PRINTED
    tmp_px = im_tmp.shape[0]
    stride = 2
    # zoom the template to the right size in pixels
    tmp = ndimage.zoom(im_tmp, target_px / tmp_px)
    a = xcorr_prep(im_warped[::stride,::stride])
    b = xcorr_prep(tmp[::stride,::stride])
    xcorr = signal.fftconvolve(
        a,
        b[::-1, ::-1],
        mode='valid'
    )
    y,x = np.unravel_index(np.argmax(xcorr), xcorr.shape)
    xfound = x + b.shape[0] / 2
    yfound = y + b.shape[1] / 2

    if ax:
        ax.imshow(a, cmap='Greys_r')
        ax.scatter(xfound, yfound, s=80, color='w')
        ax.scatter(xfound, yfound, label=template_filename)

    return xfound * stride, yfound * stride

In [23]:
def find_targets(image_dir, mtx, dist, optimal_camera_matrix, plot=False):
    '''
    After we get the camera matrix, let's use it to undo distortion in all test images,
    re-find chessboard corners, then find a homographic transform of the undistorted
    chessboard corners to the ideal, rectified plane of the chessboard (the
    "bird's-eye" view of it).
    Use that and information of how far apart the chessboard intersections really are
    to calculate a px-to-m conversion.
    Finally, do a cross-correlation to find the location of all target images, in m,
    relative to the chessboard 0,0 intersection.
    '''
    targets = ['raft_target', 'SW_target', 'SE_target', 'NW_target', 'NE_target']

    files = (
        sorted(glob.glob(os.path.join(image_dir, '**.jpeg'))) +
        sorted(glob.glob(os.path.join(image_dir, '**.jpg'))) + 
        sorted(glob.glob(os.path.join(image_dir, '**.JPG')))
    )

    image_data = {}
    px_per_ms = []
    num_steps = len(files)
    for k, file in enumerate(files):
        progress = 100. * (k) / num_steps
        print(f'Progress: {progress:.2f} %')

        # use camera cal matrix to de-distort all images
        im = cv2.imread(file)
        im = np.rot90(im, k=-1)
        undistorted_image = cv2.undistort(
            im,
            mtx,
            dist,
            None, 
            optimal_camera_matrix
        )

        # re-find chessboard corners in undistorted image
        gray = cv2.cvtColor(undistorted_image, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, CHESSBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
        print(f'Corner detection after de-distortion in {file}:', ret)
        if ret == True:
            corners2 = cv2.cornerSubPix(gray, corners, (5,5),(-1,-1), criteria)
            undistorted_image_corners = cv2.drawChessboardCorners(gray, CHESSBOARD, corners2, ret)
        else:
            continue

        # find the homographic transform that undistorts the found chessboard corners into the rectified, "bird's eye" view of the chessboard.
        this_img = corners2
        # magic number: homographic transform for de-projection sometimes makes resulting image really tiny for some reason.
        # if we didn't scale it back up, we'd have only a few pixels across each chessboard intersection, degrading the
        # quality of the subpixel refinement and eventually the template location xcorr too.
        sf = 3000.
        h, status = cv2.findHomography(
            this_img[:,0,:],
            objp[0,:,:2] * sf + corners2[0][0]
        )
        # deproject chessboard points
        corners_warped = cv2.perspectiveTransform(
            corners2,
            h
        )
        im_warped = cv2.warpPerspective(
            undistorted_image,
            h,
            (undistorted_image.shape[1], undistorted_image.shape[0])
        )

        # calc pixel scale: get pairs of euclidean distances along each axis.
        # this leaves some information on the table (e.g. expected distance info of 
        # non-adjacent pairs), but is simple and there are enough pairs that the 
        # average should be good to submm precision across ~1 m.
        found_corners = corners_warped[:,0,:].reshape((CHESSBOARD[1], CHESSBOARD[0], 2))
        dists0 = np.array([])
        for i in range(found_corners.shape[0]):
            d = np.diff(found_corners[i,:,:], axis=0)
            dists0 = np.concatenate([dists0, np.sqrt((d ** 2).sum(axis=1))])
        dists1 = np.array([])
        for j in range(found_corners.shape[1]):
            d = np.diff(found_corners[:,j,:], axis=0)
            dists1 = np.concatenate([dists1, np.sqrt((d ** 2).sum(axis=1))])
        dists = np.concatenate([dists0, dists1])
        avg_spacing = np.mean(np.abs(dists))
        std_spacing = np.std(np.abs(dists))
        px_per_m = avg_spacing / SQUARE_SIZE
        px_per_ms.append(px_per_m)

        # find pixel locs of all targets
        target_dir = os.curdir
        target_locs = {}
        target_locs['chessboard_origin'] = tuple(corners2[0][0].astype(float))

        if plot:
            plt.figure()
            ax = plt.axes()
        else:
            ax = None
        for target in targets:
            x,y = find_template(os.path.join(target_dir, target + '.png'), im_warped, px_per_m, ax=ax)
            target_locs[target] = (x,y)
        if plot:
            ax.legend(loc='best')

        m_per_inch = 0.0254
        for target in [item for item in targets if 'SW' not in item]:
            print(f'distance of {target} from SW:', np.linalg.norm(np.array(target_locs[target]) - np.array(target_locs['SW_target'])) / px_per_m / m_per_inch, 'in')

        # save off some data
        image_data[file] = {}
        image_data[file]['px_per_m'] = px_per_m
        image_data[file]['target_locs'] = target_locs

    frac_err = np.std(px_per_ms) / np.mean(px_per_ms)
    avg_px_per_m = np.mean(px_per_ms)
    image_data['source dir'] = os.path.abspath(image_dir)
    image_data['avg_px_per_m'] = avg_px_per_m

    return image_data

If you get funny results (fake ring images with data from the center around the perimeter), consider this: https://answers.opencv.org/question/28438/undistortion-at-far-edges-of-image/ lenses with nonlinear distortion parameters from the center to the edge present problems to the algorithm. apparently it can be mitigated by digital zoom/cropping out data from the corners of the sensor. This effect was very noticeable on my iPhone 8 camera. Regardless of what you do, you should probably take all photos with the same distortion parameters, i.e. same optical zoom configuration (prime lens if possible), same digital crop factor.

In [8]:
# global stuff
CHESSBOARD = (5,8)
SQUARE_SIZE = 0.02545 # m
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# the set of chessboard corner coords in the plane of the chessboard
objp = np.zeros((1, CHESSBOARD[0] * CHESSBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHESSBOARD[0], 0:CHESSBOARD[1]].T.reshape(-1, 2) * SQUARE_SIZE

# calibrate the camera for distortion
mtx, dist, optimal_camera_matrix = calibrate_camera(os.path.join('input', 'camera_cal'), plot=False)

Camera calibration corner finding in input\camera_cal\2022-03-08 10.47.59.jpg: True
Camera calibration corner finding in input\camera_cal\2022-03-08 10.48.00 (1).jpg: True
Camera calibration corner finding in input\camera_cal\2022-03-08 10.48.02.jpg: True
Camera calibration corner finding in input\camera_cal\2022-03-08 10.48.00.jpg: True
Camera calibration corner finding in input\camera_cal\2022-03-08 10.48.01.jpg: True
Camera calibration corner finding in input\camera_cal\2022-03-08 10.48.05.jpg: True
Camera calibration corner finding in input\camera_cal\2022-03-08 10.48.06.jpg: True
Camera calibration corner finding in input\camera_cal\2022-03-08 10.48.08 (1).jpg: True
Camera calibration corner finding in input\camera_cal\2022-03-08 10.48.08.jpg: True
Camera calibration corner finding in input\camera_cal\2022-03-08 10.48.07.jpg: True
Camera calibration corner finding in input\camera_cal\2022-03-08 10.48.09.jpg: True
Camera calibration corner finding in input\camera_cal\2022-03-08 10.

In [19]:
# %load_ext line_profiler

In [20]:
# %lprun -f find_targets find_targets(os.path.join("input", "targets"), mtx, dist, optimal_camera_matrix)

In [21]:
# %lprun -f find_template find_template(os.path.join("raft_target" + ".png"), cv2.imread(os.path.join("input", "targets", "2022-03-08 10.49.00.jpg")), 3000, ax=None)

In [24]:
plt.close('all')
image_data = find_targets(
    os.path.join('input', 'targets'),
    mtx,
    dist,
    optimal_camera_matrix,
    plot=True
)

Progress: 0.00 %
Corner detection after de-distortion in input\targets\2022-03-08 10.49.00.jpg: True
distance of raft_target from SW: 24.055070372200543 in
distance of SE_target from SW: 24.080258634747352 in
distance of NW_target from SW: 23.832236868378285 in
distance of NE_target from SW: 34.00294461763513 in
Progress: 3.85 %
Corner detection after de-distortion in input\targets\2022-03-08 10.49.01 (1).jpg: True
distance of raft_target from SW: 24.0540412569952 in
distance of SE_target from SW: 24.07922844194699 in
distance of NW_target from SW: 23.831217286354235 in
distance of NE_target from SW: 34.020274792740395 in
Progress: 7.69 %
Corner detection after de-distortion in input\targets\2022-03-08 10.49.01.jpg: True
distance of raft_target from SW: 24.055944862861605 in
distance of SE_target from SW: 24.081134041094796 in
distance of NW_target from SW: 23.833103258218394 in
distance of NE_target from SW: 34.00418075026981 in
Progress: 11.54 %
Corner detection after de-distortion i

C:\Users\trump\AppData\Local\Temp/ipykernel_14192/2939710291.py:93: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


distance of raft_target from SW: 24.091795455687098 in
distance of SE_target from SW: 24.106102219826784 in
distance of NW_target from SW: 23.831750120543287 in
distance of NE_target from SW: 34.0395096564416 in
Progress: 80.77 %
Corner detection after de-distortion in input\targets\2022-03-08 10.49.05.jpg: True
distance of raft_target from SW: 24.054541884031863 in
distance of SE_target from SW: 24.054341421521524 in
distance of NW_target from SW: 23.831713275845683 in
distance of NE_target from SW: 34.00219757538363 in
Progress: 84.62 %
Corner detection after de-distortion in input\targets\2022-03-08 10.49.06 (1).jpg: True
distance of raft_target from SW: 24.036239974576414 in
distance of SE_target from SW: 24.054002855378812 in
distance of NW_target from SW: 23.805968443397823 in
distance of NE_target from SW: 14.675554869293023 in
Progress: 88.46 %
Corner detection after de-distortion in input\targets\2022-03-08 10.49.06.jpg: True
distance of raft_target from SW: 24.05486042354003 

In [29]:
outlier_thresh = 0.01

# meas dist from SW corner to each target
actuals = {
    'raft_target': 0.61198125
}

for target in ['raft_target', 'SE_target', 'NW_target', 'NE_target']:
    query_meas = []
    for img in image_data.keys():
        if not isinstance(image_data[img], dict):
            continue
        query_raft = np.linalg.norm(
            np.array(image_data[img]['target_locs'][target]) - 
            np.array(image_data[img]['target_locs']['SW_target'])
        ) / image_data[img]['px_per_m']
        query_meas.append(query_raft)

    # clean data
    query_meas = [item for item in query_meas if np.median(query_meas) - item < outlier_thresh]
    
    query_meas = np.array(query_meas)
    query_raft_mean = query_meas.mean()
    mean_inch = query_raft_mean / 0.0254
    mean_inch_frac = (mean_inch - np.floor(mean_inch)) / (1./32.)
    query_raft_std = query_meas.std()
    
    print(f'dist of {target} from SW\n    mean {query_raft_mean}\n    std {query_raft_std}\n    std (mm) {query_raft_std * 1000}\n    % err {100. * query_raft_std / query_raft_mean}')
    print(f'{np.floor(mean_inch):.0f} + {mean_inch_frac:.2f}/32\"')
    
    plt.figure()
    ax = plt.axes()
    ax.set_title(f'{target} distance from SW target (L2 norm) (m)\nN={len(query_meas)}')
    # ax.hist(query_meas, bins=20)
    sns.kdeplot(query_meas, shade=True, color='k', label='Kernel Density Estimate')
    sns.rugplot(query_meas, color='k')
    ax.axvline(query_raft_mean, color='k', label=f'Mean ~{query_raft_mean:.4f} m')
    ax.axvline(query_raft_mean - query_raft_std, color='k', linestyle='--')
    ax.axvline(query_raft_mean + query_raft_std, color='k', linestyle='--', label=f'1 std. dev. ~{query_raft_std * 1000:.1f} mm')
    if target in actuals.keys():
        ax.axvline(actuals[target], color='r', label=f'Actual (1/16" graduated tape measure): {actuals[target]:.4f} m')
    ax.grid(True)
    ax.legend(loc='best')
    ax.set_xlabel('Distance (m)')
    ax.set_ylabel('Probability Density')


dist of raft_target from SW
    mean 0.6109246438370295
    std 0.0003563155694079216
    std (mm) 0.3563155694079216
    % err 0.05832398037997179
24 + 1.67/32"
dist of SE_target from SW
    mean 0.6115370900753268
    std 0.00034313432719845945
    std (mm) 0.34313432719845943
    % err 0.056110141603380674
24 + 2.44/32"
dist of NW_target from SW
    mean 0.6051328802447499
    std 0.00029801232023136097
    std (mm) 0.298012320231361
    % err 0.049247418205209406
23 + 26.37/32"
dist of NE_target from SW
    mean 0.8638498086698068
    std 0.0004463064683132817
    std (mm) 0.4463064683132817
    % err 0.05166482226818151
34 + 0.31/32"
